---
title: SpatioTemporal Field - T2M - Stationary
subject: AEMET T2M
short_title: d - Region Analysis
authors:
  - name: J. Emmanuel Johnson
    affiliations:
      - CSIC
      - UCM
      - IGEO
    orcid: 0000-0002-6739-0053
    email: juanjohn@ucm.es
license: CC-BY-4.0
keywords: notation
---

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "" # first gpu
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'FALSE'

import jax
jax.config.update('jax_platform_name', 'cpu')

import numpyro
import multiprocessing

num_devices = multiprocessing.cpu_count()
numpyro.set_platform("cpu")
numpyro.set_host_device_count(num_devices)

In [ ]:
import autoroot
from pathlib import Path
import numpy as np
import xarray as xr
import pandas as pd
import pint_xarray
import arviz as az

from st_evt import CANDIDATE_STATIONS
from st_evt.utils import ExperimentPath
from st_evt.viz import (
    plot_scatter_ts,
    plot_histogram,
    plot_density,
    plot_return_level_gevd_manual_unc_multiple,
    plot_periods,
    plot_periods_diff,
    plot_spain,
    plot_density_multiple
)
from st_evt.extremes import estimate_return_level_gevd, calculate_exceedence_probs
from omegaconf import OmegaConf

import jax
import jax.random as jrandom
import jax.numpy as jnp
import pandas as pd

rng_key = jrandom.PRNGKey(123)

from numpyro.infer import Predictive
import arviz as az

import numpyro
from numpyro.diagnostics import hpdi
import numpyro.distributions as dist
from numpyro import handlers
from numpyro.infer import MCMC, NUTS
import xarray as xr
import regionmask

import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, FuncFormatter
from st_evt._src.modules.models.aemet import utils_region
import seaborn as sns
from loguru import logger
sns.reset_defaults()
sns.set_context(context="talk", font_scale=0.7)


%config InlineBackend.figure_format = 'retina'
plt.style.use(
    "https://raw.githubusercontent.com/ClimateMatchAcademy/course-content/main/cma.mplstyle"
)


%matplotlib inline
%load_ext autoreload
%autoreload 2

## Data

In [ ]:
# results_data_path = "/home/juanjohn/pool_data/dynev4eo/temp/scratch_pipelines/results/az_nonstationary_gp_mcmc_redfeten.zarr"
# figures_path = Path("/home/juanjohn/pool_data/dynev4eo/temp/scratch_pipelines/figures/nonstationary_gp_mcmc_redfeten/spain")

In [ ]:
results_root_path = "/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/stationary_iid_mcmc_redfeten/results"
results_file_name = "stationary_iid_mcmc_redfeten.zarr"
results_data_path = Path(results_root_path).joinpath(results_file_name)

figures_path = Path(results_root_path).joinpath("figures/spain")

### MCMC Results

In [ ]:
variable = "t2max"
covariate = "gmst"

## Model Errors - Posterior

### Negative Log-Likelihood

In [ ]:
PLOT_VARIABLE = "nll"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 5_000

In [ ]:
az_ds = az.from_zarr(str(results_data_path))
data_results = az.extract(az_ds, group="log_likelihood", num_samples=num_samples).median(dim=["sample"]).load()
data_results = data_results.sortby(covariate)
idata = data_results[VARIABLE].rename(PLOT_VARIABLE).sum(dim=covariate)
idata = idata.where(idata.red_feten_mask == 1, drop=True)

In [ ]:
subfigures_path = Path(figures_path).joinpath(f"{PLOT_VARIABLE}/posterior")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_static_spatial_variable(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[PLOT_VARIABLE],
    cmap="Reds_r",
    bounds = (-160, 10, -50)
)

## Model Errors - Posterior Predictive

### I - Negative Log-Likelihood

In [ ]:
PLOT_VARIABLE = "nll"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 5_000

In [ ]:
az_ds = az.from_zarr(store=str(results_data_path))
data_results = az.extract(az_ds, group="posterior_predictive", num_samples=num_samples).median(dim=["sample"]).load()
data_results = data_results.sortby(covariate)
idata = data_results["nll"].rename(PLOT_VARIABLE).sum(dim=covariate)

In [ ]:
subfigures_path = Path(figures_path).joinpath(f"{PLOT_VARIABLE}/posterior_predictive")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_static_spatial_variable_redfeten(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[PLOT_VARIABLE],
    cmap="Reds_r",
    bounds = (-160, 10, -50)
)

### II - Residuals

In [ ]:
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score

In [ ]:
PLOT_VARIABLE = "residuals"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 5_000

In [ ]:
data_results = az.extract(az_ds, group="posterior_predictive", num_samples=num_samples).median(dim=["sample"]).load()
data_results = data_results.sortby("gmst")
y_pred = data_results[variable].rename("y_pred")
y_true = data_results[f"{variable}_true"].rename("y_true")
idata = (y_true - y_pred).mean(dim=[covariate]).load().rename(PLOT_VARIABLE)

In [ ]:
subfigures_path = Path(figures_path).joinpath(f"{PLOT_VARIABLE}/posterior_predictive")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_static_spatial_variable_redfeten(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[PLOT_VARIABLE],
    cmap="Reds_r",
    bounds = (-7.5, 10, 7.5)
)

### III - Absolute Residuals

In [ ]:
PLOT_VARIABLE = "residuals_abs"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 5_000

In [ ]:
data_results = az.extract(az_ds, group="posterior_predictive", num_samples=num_samples).median(dim=["sample"]).load()
data_results = data_results.sortby("gmst")
y_pred = data_results[variable].rename("y_pred")
y_true = data_results[f"{variable}_true"].rename("y_true")
idata = np.abs(y_true - y_pred).mean(dim=[covariate]).load().rename(PLOT_VARIABLE)

In [ ]:
subfigures_path = Path(figures_path).joinpath(f"{PLOT_VARIABLE}/posterior_predictive")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_static_spatial_variable_redfeten(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[PLOT_VARIABLE],
    cmap="Reds",
    bounds = (0.0, 10, 7.5)
)

## Model Errors - All

### Negative Log-Likelihood - All

In [ ]:
subfigures_path = figures_path.joinpath("posterior_predictive/nll")
subfigures_path.mkdir(parents=True, exist_ok=True)

#### Posterior

In [ ]:
variable = "t2max"
az_ds = az.from_zarr(store=str(results_data_path))
idata = az.extract(az_ds, group="log_likelihood", num_samples=4000).median(dim=["sample"]).load()
isub_post = idata.sortby("gmst")[variable].rename("nll").sum(dim=covariate)

#### Posterior - Predictive

In [ ]:

az_ds = az.from_zarr(store=str(results_data_path))
idata = az.extract(az_ds, group="posterior_predictive", num_samples=4000).median(dim=["sample"]).load()
isub_postpred = idata.sortby("gmst")["nll"].sum(dim=covariate)

In [ ]:
# labels
cbar_label = "Negative Log Predictive Density\n"
cbar_label += r"$\boldsymbol{L}(\mathbf{y};\boldsymbol{\theta},\boldsymbol{\phi},\mathcal{D})$"

#### Density

In [ ]:
returns = []

returns.append({
    "period": "Red Feten (Posterior)",
    "color": "gray",
    "values":  isub_post.values.ravel(),
    "linestyle": "-",
    "values_units": "", # "[mm/day]",
})

returns.append({
    "period": "Red Feten (Posterior Predictive)",
    "color": "tab:blue",
    "values":  isub_postpred.where(isub_postpred.red_feten_mask == 1, drop=True).values.ravel(),
    "linestyle": "-",
    "values_units": "", # "[mm/day]",
})
returns.append({
    "period": "Not Red Feten (Posterior Predictive)",
    "color": "tab:red",
    "linestyle": "--",
    "values":  isub_postpred.where(isub_postpred.red_feten_mask == 0, drop=True).values.ravel(),
    "values_units": "", # "[mm/day]",
})

In [ ]:
fig, ax = plot_density_multiple(returns, log_bins=False)
fig.set_size_inches(6, 5)
# fig.set_size_inches(8, 4.5)
ax.set(
    xlabel=cbar_label,
)

formatter = ScalarFormatter()
formatter.set_scientific(False)
ax.xaxis.set_major_formatter(formatter)
plt.legend(fontsize=10, loc="upper left")
plt.tight_layout()
fig.savefig(subfigures_path.joinpath("density_all.png"))
plt.show()

## Global Parameters

### Concentration - Posterior

In [ ]:
PLOT_VARIABLE = "concentration"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 5_000

In [ ]:
az_ds = az.from_zarr(store=str(results_data_path))
idata = az.extract(az_ds, group="posterior", num_samples=num_samples)
idata = idata.sortby("gmst")
idata = data_results[PLOT_VARIABLE]

In [ ]:
subfigures_path = Path(figures_path).joinpath(f"{PLOT_VARIABLE}/posterior")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_static_spatial_variable(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[f"{PLOT_VARIABLE}_iid"],
    cmap="Reds",
    bounds = (-1, 10, 0.0)
)

### Concentration - Posterior Predictive

In [ ]:
PLOT_VARIABLE = "concentration"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 5_000

In [ ]:
az_ds = az.from_zarr(store=str(results_data_path))
idata = az.extract(az_ds, group="posterior_predictive", num_samples=num_samples)
idata = idata.sortby("gmst")
idata = data_results[PLOT_VARIABLE]

In [ ]:
subfigures_path = Path(figures_path).joinpath(f"{PLOT_VARIABLE}/posterior_predictive")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_static_spatial_variable(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[f"{PLOT_VARIABLE}_iid"],
    cmap="Reds",
    bounds = (-1, 10, 0.0)
)

### Scale - Posterior

In [ ]:
PLOT_VARIABLE = "scale"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 5_000

In [ ]:
az_ds = az.from_zarr(store=str(results_data_path))
idata = az.extract(az_ds, group="posterior", num_samples=num_samples)
idata = idata.sortby("gmst")
idata = data_results[PLOT_VARIABLE]

In [ ]:
subfigures_path = Path(figures_path).joinpath(f"{PLOT_VARIABLE}/posterior")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_static_spatial_variable(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[f"{PLOT_VARIABLE}_iid"],
    cmap="Reds",
    bounds = (0.5, 10, 3.0)
)

### Scale - Posterior Predictive

In [ ]:
PLOT_VARIABLE = "scale"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 5_000

In [ ]:
az_ds = az.from_zarr(store=str(results_data_path))
idata = az.extract(az_ds, group="posterior_predictive", num_samples=num_samples)
idata = idata.sortby("gmst")
idata = data_results[PLOT_VARIABLE]

In [ ]:
subfigures_path = Path(figures_path).joinpath(f"{PLOT_VARIABLE}/posterior_predictive")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_static_spatial_variable(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[f"{PLOT_VARIABLE}_iid"],
    cmap="Reds",
    bounds = (0.5, 10, 3.0)
)

### Location - Posterior

In [ ]:
PLOT_VARIABLE = "location"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 5_000

In [ ]:
az_ds = az.from_zarr(store=str(results_data_path))
idata = az.extract(az_ds, group="posterior", num_samples=num_samples)
idata = idata.sortby("gmst")
idata = data_results[PLOT_VARIABLE]

In [ ]:
subfigures_path = Path(figures_path).joinpath(f"{PLOT_VARIABLE}/posterior")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_static_spatial_variable(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[f"{PLOT_VARIABLE}_iid"],
    cmap="Reds",
    bounds = (20.0, 10, 45.0)
)

### Location - Posterior Predictive

In [ ]:
PLOT_VARIABLE = "location"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 5_000

In [ ]:
az_ds = az.from_zarr(store=str(results_data_path))
idata = az.extract(az_ds, group="posterior_predictive", num_samples=num_samples)
idata = idata.sortby("gmst")
idata = data_results[PLOT_VARIABLE]

In [ ]:
subfigures_path = Path(figures_path).joinpath(f"{PLOT_VARIABLE}/posterior_predictive")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_static_spatial_variable(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[f"{PLOT_VARIABLE}_iid"],
    cmap="Reds",
    bounds = (20.0, 10, 45.0)
)

### 100-Year Return Period - Posterior

In [ ]:
PLOT_VARIABLE = "return_level_100"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 5_000

In [ ]:
data_results = az.extract(az_ds, group="posterior", num_samples=num_samples)
data_results = data_results.sortby("gmst")
data_results = utils_region.calculate_ds_return_periods(data_results, ["sample"])
idata = data_results[PLOT_VARIABLE].median(dim=["sample"]).load()
# idata = idata.where(idata.red_feten_mask == 1, drop=True)

In [ ]:
subfigures_path = Path(figures_path).joinpath(f"{PLOT_VARIABLE}/posterior")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_static_spatial_variable(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[f"{PLOT_VARIABLE}_spatial"],
    cmap="Reds",
    bounds = (25, 10, 50),
)

### 100-Year Return Period - Posterior Predictive

In [ ]:
PLOT_VARIABLE = "return_level_100"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 5_000

In [ ]:
data_results = az.extract(az_ds, group="posterior_predictive", num_samples=num_samples)
data_results = data_results.sortby("gmst")
data_results = utils_region.calculate_ds_return_periods(data_results, ["sample"])
idata = data_results[PLOT_VARIABLE].median(dim=["sample"]).load()

In [ ]:
subfigures_path = Path(figures_path).joinpath(f"{PLOT_VARIABLE}/posterior_predictive")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_static_spatial_variable(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[f"{PLOT_VARIABLE}_spatial"],
    cmap="Reds",
    bounds = (25, 10, 50),
)